In [1]:
from selenium import webdriver
import time
import pandas as pd
from datetime import datetime
import random
from tweet_class import Tweet
import re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import uuid
import logging
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [2]:
#url = "https://x.com/search?f=live&q=%28from%3ARER_A%29+%28to%3ARER_A%29+until%3A2024-12-03&src=typed_query"
url = 'https://x.com/RER_A/status/1802654385207480781'

In [3]:
data_dir="/home/seluser/twitter_scraper/selenium"
options = webdriver.ChromeOptions()
options.add_argument("--mute-audio")
options.add_experimental_option("detach", True)
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument(f"--user-data-dir={data_dir}")
options.add_argument("--disable-blink-features=AutomationControlled") 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_experimental_option("useAutomationExtension", False) 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")


In [4]:
chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
chrome.get(url)

In [5]:
already_done_set = set()
already_done_set_with_threadid = {}

In [14]:
def _scrap_thread(chrome, thread_id):
        """
        Scrap all the tweet in a thread belonging to the account mentioned in the constructor
        A thread is a tweet that has been replied to by other tweet

        To check if a tweet in the last from the thread it is either the last tweet of the thread or is followed by a separator
        Args:
            chrome (webdriver): The webdriver to use
            thread_id (str): The ID of the thread to link all the tweet in the thread
        
        Returns:
            None: If everything works, nothing is returned otherwise an error is raised
        """
        subtweets = chrome.find_elements('xpath', '//article[@data-testid="tweet"]')
        for index, subtweet in enumerate(subtweets):
                print(subtweet.text)
                #last_thread_tweet = subtweet.find_elements('xpath', '../../../div/div/div//text()')
                try:
                    last_thread_tweet = subtweet.find_elements('xpath', './parent::div/parent::div/parent::div/following-sibling::div')[0]


                    elements = subtweet.find_elements('xpath', './div/div/div')
                    content = elements[-1].find_elements('xpath', './div[2]/div')
                    header = content[0]
                except:
                    print("continue")
                    continue
                
                # Check if the tweet is from the account mentioned in the constructor
                if 'RER_A' not in header.text:
                    return thread_id

                try:
                    if index==0:
                        time_posted = scrap_tweet(subtweet, 'Normal')
                    else:
                        time_posted = scrap_tweet(subtweet, 'Réponse')
                except Exception as e:
                    logging.error("Error while scrapping tweet in thread")
                    logging.error(f"Thread link :{chrome.current_url}, reponse index : {index}")   
                    continue
                

                if time_posted in already_done_set:
                    thread_id = already_done_set_with_threadid.get(time_posted)
                already_done_set.add(time_posted)
                already_done_set_with_threadid[time_posted] = thread_id
 
                #if last_thread_tweet == "" or index == len(subtweets) - 1:
                #    return
                if last_thread_tweet.text == "" or index == len(subtweets) - 1:
                    return thread_id
        return thread_id

def scrap_tweet(tweet, type):
    """
    Scrap a tweet and return a Tweet object with all the information retrieved
    There a special cae when the tweet is a quote tweet, in this case, the tweet content correspond to the date of the quoted tweet
    Args:
        tweet (WebElement): The tweet to scrap
        type (str): The type of tweet (Normal or Réponse)
    Returns:
        Tweet: The tweet object with all the information retrieved
    """

    try:
        elements = tweet.find_elements('xpath', './div/div/div[2]')
        content = elements[-1].find_elements('xpath', './div[2]/div')
    except Exception as e:
        raise "Couldn't get tweet content"
    
    header = content[0]
    time_posted = header.find_element('xpath', './/time').get_attribute('datetime')
    time_posted_conv = datetime.strptime(time_posted, "%Y-%m-%dT%H:%M:%S.%fZ")

    
    

    # Check if the tweet is a quote tweet
    try:
        date_cited_tweet = content[-2].find_elements('xpath','.//time')[0].get_attribute('datetime')
        time_posted_conv_2 = datetime.strptime(date_cited_tweet, "%Y-%m-%dT%H:%M:%S.%fZ")
        print('citation')
        return time_posted_conv
    except Exception as e:
        pass

    try:
        content_tweet = content[-2]
        #print(content_tweet.text)

        elements = content_tweet.find_elements(By.XPATH, "./div//*")
        content_list = []

        for element in elements:
            if element.tag_name == 'span':
                content_list.append(element.text)
            elif element.tag_name == 'img':
                content_list.append(element.get_attribute('alt'))
        #print(content_list)
    except:
        raise("Aie")


    return time_posted_conv

In [15]:
_scrap_thread(chrome, "000000")

RER A
@RER_A
Suivre
Le train actuellement à Châtelet–Les Halles et en direction de Saint-Germain-en-Laye stationne en raison d'une gêne à la fermeture des portes. #RERA
12:47 PM · 17 juin 2024
·
13,4 k
 vues
2
1
6
continue
RER A
@RER_A
·
17 juin
 Le trafic est très perturbé de Marne-la-Vallée–Chessy • Boissy-Saint-Léger vers Cergy-Le Haut • Poissy et Saint-Germain-en-Laye en raison d'un train en panne à Châtelet–Les Halles. #RERA
2
1
7
4 k
RER A
@RER_A
·
17 juin
 Le trafic est perturbé de Marne-la-Vallée–Chessy • Boissy-Saint-Léger vers Cergy-Le Haut • Poissy et Saint-Germain-en-Laye en raison d'un train en panne à Châtelet–Les Halles. #RERA
1
2
5 k
RER A
@RER_A
·
17 juin
 Le trafic est rétabli  de Marne-la-Vallée–Chessy • Boissy-Saint-Léger vers Cergy-Le Haut • Poissy et Saint-Germain-en-Laye (train en panne). #RERA
1
3 k


'000000'

In [7]:
tweets_on_page = chrome.find_elements('xpath', '//article[@data-testid="tweet"]')
main_page = chrome.window_handles[0]
for tweet in tweets_on_page:


    thread_id = str(uuid.uuid4())

    # Scrap the main tweet informations
    try:
        time_posted_conv = scrap_tweet(tweet, 'Réponse')
    except Exception as e:
        logging.error("Error while scrapping main tweet")
        break
    
    
    print("-------------------")
    print(time_posted_conv)
    print(already_done_set)
    print(time_posted_conv in already_done_set)
    print(already_done_set_with_threadid)
    print("-------------------")
    
    
    # Check if the tweet has already been scrapped
    if time_posted_conv in already_done_set:
        continue
    already_done_set.add(time_posted_conv)



    # Click sur le tweet si adresse est sur X sinon rien
    # Click on the tweet to access the thread
    try:
        #tweetContent = tweet.find_element('xpath', './/div[@data-testid="tweetText"]')
        tweetThreadLink = tweet.find_element("xpath", './/a[contains(@href, "/status/")]')
        ActionChains(chrome).key_down(Keys.CONTROL).click(tweetThreadLink).key_up(Keys.CONTROL).perform()
        chrome.switch_to.window(chrome.window_handles[-1])
    except Exception as e:
        print(e)
        print("No CLICK")
        continue

    time.sleep(5 + random.randint(5, 10))

    # Scrap the thread
    try:
        res = _scrap_thread(chrome, thread_id)
        if res != 0:
            thread_id = res
    except Exception as e:
        logging.error("Error while scrapping thread")
        chrome.close()  
        chrome.switch_to.window(main_page) 
        raise e
    

    already_done_set_with_threadid[time_posted_conv] = thread_id



    #time.sleep(delay_before_closing_tab + random.randint(4, 7))
        
    # Close the new tab
    chrome.close()

    # Switch back to the original tab
    chrome.switch_to.window(main_page) 
    WebDriverWait(chrome, 10).until(
            EC.presence_of_element_located((By.XPATH, "//article[@data-testid='tweet']"))
    )
    time.sleep(5 + random.randint(5, 10))


ERROR:root:Error while scrapping main tweet
